# Análisis Comparativo de Diseño de Tuberías según Códigos ASME B31.X

## ASME B31.1:2022

In [38]:
from math import sqrt
from math import pi, sqrt
import ipywidgets as widgets
from IPython.display import display, Markdown

### CÁLCULO DEL ESPESOR REQUERIDO

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

# === Valores por defecto ===
defaults = {
    "P": 2.0,
    "D_o": 219.1,
    "S": 120.0,
    "E": 1.0,
    "W": 1.0,
    "Y": 0.4,
    "A": 1.5,
    "t_n": 8.18,
    "tol": 0.125
}

# === Widgets de entrada ===
P    = widgets.FloatSlider(value=defaults["P"], min=0.1, max=10.0, step=0.1, description='P [MPa]')
D_o  = widgets.FloatSlider(value=defaults["D_o"], min=50, max=600, step=1, description='D_o [mm]')
S    = widgets.FloatSlider(value=defaults["S"], min=10, max=300, step=1, description='S [MPa]')
E    = widgets.FloatSlider(value=defaults["E"], min=0.6, max=1.0, step=0.05, description='E')
W    = widgets.FloatSlider(value=defaults["W"], min=0.5, max=1.0, step=0.05, description='W')
Y    = widgets.FloatSlider(value=defaults["Y"], min=0.0, max=0.8, step=0.05, description='Y')
A    = widgets.FloatText(value=defaults["A"], description='A [mm]')
t_n  = widgets.FloatText(value=defaults["t_n"], description='t_n [mm]')
tol  = widgets.FloatSlider(value=defaults["tol"], min=0.0, max=0.3, step=0.01, description='Tol')

# Botón reset
reset_btn = widgets.Button(description="🔄 Resetear valores", button_style="warning")

# === Función de reset ===
def reset_values(b):
    P.value    = defaults["P"]
    D_o.value  = defaults["D_o"]
    S.value    = defaults["S"]
    E.value    = defaults["E"]
    W.value    = defaults["W"]
    Y.value    = defaults["Y"]
    A.value    = defaults["A"]
    t_n.value  = defaults["t_n"]
    tol.value  = defaults["tol"]

reset_btn.on_click(reset_values)

# Mostrar controles + botón
display(P, D_o, S, E, W, Y, A, t_n, tol, reset_btn)

# === Función de cálculo ===
def calcular(P, D_o, S, E, W, Y, A, t_n, tol):
    P_si    = P * 1e6
    D_o_si  = D_o / 1000
    S_si    = S * 1e6
    A_si    = A / 1000
    t_n_si  = t_n / 1000

    numerador   = P_si * D_o_si
    denominador = 2 * (S_si * E * W + P_si * Y)
    t_min_si    = (numerador / denominador) + A_si  # [m]

    t_disp_si = t_n_si * (1 - tol)                  # [m]

    t_min  = t_min_si * 1000
    t_disp = t_disp_si * 1000
    criterio = "✅ Cumplido" if t_disp_si >= t_min_si else "❌ NO Cumplido"

    html = f"""
    <div style="border:2px solid #004080; border-radius:10px; padding:15px; background:#f9f9f9; font-family:Arial;">
        <h3 style="color:#004080;">📊 Resultados del Cálculo</h3>
        <ul style="font-size:15px; line-height:1.6;">
            <li>📏 <b>Espesor mínimo requerido:</b> {t_min:.2f} mm</li>
            <li>📐 <b>Espesor disponible:</b> {t_disp:.2f} mm</li>
        </ul>
        <p style="font-size:16px;"><b>Criterio de aceptación:</b> {criterio}</p>
    </div>
    """
    display(HTML(html))

# === Conectar widgets ===
out = widgets.interactive_output(
    calcular, {"P": P, "D_o": D_o, "S": S, "E": E, "W": W, "Y": Y, "A": A, "t_n": t_n, "tol": tol}
)
display(out)

FloatSlider(value=2.0, description='P [MPa]', max=10.0, min=0.1)

FloatSlider(value=219.1, description='D_o [mm]', max=600.0, min=50.0, step=1.0)

FloatSlider(value=120.0, description='S [MPa]', max=300.0, min=10.0, step=1.0)

FloatSlider(value=1.0, description='E', max=1.0, min=0.6, step=0.05)

FloatSlider(value=1.0, description='W', max=1.0, min=0.5, step=0.05)

...

FloatText(value=1.5, description='A [mm]')

FloatText(value=8.18, description='t_n [mm]')

FloatSlider(value=0.125, description='Tol', max=0.3, step=0.01)

Button(button_style='warning', description='🔄 Resetear valores', style=ButtonStyle())

Output()

### CÁLCULO DE ESFUERZOS

In [40]:
# %% [markdown]
# # ⚙️ Cálculo de Esfuerzos Combinados en Tuberías
# Basado en ASME B31.3 – Ecuaciones 15, 16 y 17

# %%
import ipywidgets as widgets
from IPython.display import display, HTML
from math import sqrt

# === Valores por defecto ===
defaults = {
    "P": 2.0,      # MPa
    "D_o": 219.1,  # mm
    "t_n": 8.18,   # mm
    "A_p": 0.005,  # m²
    "Z": 0.0002756,
    "Z_t": 0.0005511,
    "S_h": 120.0,  # MPa
    "S_c": 138.0,  # MPa
    # Cargas sostenidas
    "F_a": 0.0, "M_iA": 3000.0, "M_oA": 1500.0, "M_tA": 0.0,
    # Cargas ocasionales
    "F_b": 0.0, "M_iB": 8000.0, "M_oB": 3500.0, "M_tB": 0.0,
    # Cargas de desplazamiento
    "F_c": 0.0, "M_iC": 12000.0, "M_oC": 6000.0, "M_tC": 0.0,
    "N": 1e6,
    # Índices de esfuerzo
    "I_i": 1.0, "I_o": 1.0, "I_t": 1.0,
    "i_a": 1.0, "i_l": 1.0, "i_o": 1.0, "i_t": 1.0
}

# === Widgets de entrada ===
P     = widgets.FloatSlider(value=defaults["P"], min=0.1, max=10.0, step=0.1, description="P [MPa]")
D_o   = widgets.FloatSlider(value=defaults["D_o"], min=50, max=600, step=1, description="D_o [mm]")
t_n   = widgets.FloatSlider(value=defaults["t_n"], min=1, max=30, step=0.1, description="t_n [mm]")
A_p   = widgets.FloatText(value=defaults["A_p"], description="A_p [m²]")
Z     = widgets.FloatText(value=defaults["Z"], description="Z [m³]")
Z_t   = widgets.FloatText(value=defaults["Z_t"], description="Z_t [m³]")
S_h   = widgets.FloatText(value=defaults["S_h"], description="S_h [MPa]")
S_c   = widgets.FloatText(value=defaults["S_c"], description="S_c [MPa]")

# Botón reset
reset_btn = widgets.Button(description="🔄 Resetear valores", button_style="warning")
def reset_values(b):
    P.value, D_o.value, t_n.value = defaults["P"], defaults["D_o"], defaults["t_n"]
    A_p.value, Z.value, Z_t.value = defaults["A_p"], defaults["Z"], defaults["Z_t"]
    S_h.value, S_c.value          = defaults["S_h"], defaults["S_c"]
reset_btn.on_click(reset_values)

# Mostrar controles
display(P, D_o, t_n, A_p, Z, Z_t, S_h, S_c, reset_btn)

# === Función de cálculo (robusta contra KeyError) ===
def calcular(P, D_o, t_n, A_p, Z, Z_t, S_h, S_c):
    # Conversión a SI
    P_si   = P * 1e6        # Pa
    D_o_si = D_o / 1000     # m
    t_si   = t_n / 1000     # m

    # Longitudinal por presión
    S_lp = (P_si * D_o_si) / (4 * t_si)  # Pa

    # === Ecuación 15: Sostenido ===
    F_a   = defaults.get("F_a", 0.0)
    I_i   = defaults.get("I_i", 1.0)
    I_o   = defaults.get("I_o", 1.0)
    I_t   = defaults.get("I_t", 1.0)
    M_iA  = defaults.get("M_iA", 0.0)
    M_oA  = defaults.get("M_oA", 0.0)
    M_tA  = defaults.get("M_tA", 0.0)

    S_ax          = (F_a / A_p) + S_lp        # Pa
    M_in_plane    = (I_i * M_iA) / Z          # Pa
    M_out_of_plane= (I_o * M_oA) / Z          # Pa
    M_torsional   = (I_t * M_tA) / Z_t        # Pa
    S_sost_Pa     = sqrt(S_ax**2 + M_in_plane**2 + M_out_of_plane**2 + 3*M_torsional**2)
    S_sost        = S_sost_Pa / 1e6           # MPa
    criterio_15   = "✅ Cumplido" if S_sost <= S_h else "❌ NO Cumplido"

    # === Ecuación 16: Ocasional ===
    F_b   = defaults.get("F_b", 0.0)
    M_iB  = defaults.get("M_iB", 0.0)
    M_oB  = defaults.get("M_oB", 0.0)
    M_tB  = defaults.get("M_tB", 0.0)

    S_bx    = (F_b / A_p) + S_lp
    M_iB_Pa = (I_i * M_iB) / Z
    M_oB_Pa = (I_o * M_oB) / Z
    M_tB_Pa = (I_t * M_tB) / Z_t
    S_oc_Pa = sqrt(S_bx**2 + M_iB_Pa**2 + M_oB_Pa**2 + 3*M_tB_Pa**2)
    S_oc    = S_oc_Pa / 1e6
    criterio_16 = "✅ Cumplido" if S_oc <= 1.15 * S_h else "❌ NO Cumplido"

    # === Ecuación 17: Desplazamiento ===
    i_a  = defaults.get("i_a", 1.0)
    i_l  = defaults.get("i_l", 1.0)
    i_o_ = defaults.get("i_o", 1.0)  # evitar sombrear i_o
    i_t_ = defaults.get("i_t", 1.0)

    F_c  = defaults.get("F_c", 0.0)
    M_iC = defaults.get("M_iC", 0.0)
    M_oC = defaults.get("M_oC", 0.0)
    M_tC = defaults.get("M_tC", 0.0)
    N    = defaults.get("N", 1e6)

    F_disp_Pa = (i_a * F_c) / A_p
    M_l_disp  = (i_l * M_iC) / Z
    M_o_disp  = (i_o_ * M_oC) / Z
    M_t_disp  = (i_t_ * M_tC) / Z_t
    S_des_Pa  = sqrt(F_disp_Pa**2 + M_l_disp**2 + M_o_disp**2 + 3*M_t_disp**2)
    S_des     = S_des_Pa / 1e6

    f   = min(1.0, 6.0 / (N**0.2))
    S_A = f * (1.25 * S_c + 0.25 * S_h)  # MPa
    criterio_17 = "✅ Cumplido" if S_des <= S_A else "❌ NO Cumplido"

    # === Mostrar resultados en HTML ===
    html = f"""
    <div style="border:2px solid #004080; border-radius:10px; padding:16px; background:#f9f9f9; font-family:Arial, sans-serif;">
        <h3 style="color:#004080; margin-top:0;">📊 Resultados de Esfuerzos</h3>
        <ul style="font-size:15px; line-height:1.6; margin:0 0 8px 18px;">
            <li>🟦 <b>Sostenido (Eq. 15):</b> {S_sost:.2f} MPa → {criterio_15}</li>
            <li>🟨 <b>Ocasional (Eq. 16):</b> {S_oc:.2f} MPa → {criterio_16}</li>
            <li>🟥 <b>Desplazamiento (Eq. 17):</b> {S_des:.2f} MPa → {criterio_17}</li>
        </ul>
    </div>
    """
    display(HTML(html))

# === Conectar widgets ===
out = widgets.interactive_output(
    calcular, {"P": P, "D_o": D_o, "t_n": t_n, "A_p": A_p, "Z": Z, "Z_t": Z_t, "S_h": S_h, "S_c": S_c}
)
display(out)

FloatSlider(value=2.0, description='P [MPa]', max=10.0, min=0.1)

FloatSlider(value=219.1, description='D_o [mm]', max=600.0, min=50.0, step=1.0)

FloatSlider(value=8.18, description='t_n [mm]', max=30.0, min=1.0)

FloatText(value=0.005, description='A_p [m²]')

FloatText(value=0.0002756, description='Z [m³]')

FloatText(value=0.0005511, description='Z_t [m³]')

FloatText(value=120.0, description='S_h [MPa]')

FloatText(value=138.0, description='S_c [MPa]')

Button(button_style='warning', description='🔄 Resetear valores', style=ButtonStyle())

Output()

### CÁLCULO DE SOPORTES (SPAN)

In [41]:
# %% [markdown]
# # 🧮 Cálculo de Span Máximo entre Soportes en Tuberías
# Según ASME B31.3 – Criterios de esfuerzo y deformación

# %%
import ipywidgets as widgets
from IPython.display import display, HTML
from math import pi, sqrt

# === Valores por defecto ===
defaults = {
    "D_o": 219.1,       # mm
    "t_n": 8.18,        # mm
    "rho_steel": 7850,  # kg/m³
    "rho_fluid": 1000,  # kg/m³
    "g": 9.81,          # m/s²
    "E_GPa": 200.0,     # GPa
    "S_h_MPa": 120.0,   # MPa
    "delta_adm": 10.0,  # mm
    "w_insul": 0.0      # N/m
}

# === Widgets de entrada ===
D_o        = widgets.FloatSlider(value=defaults["D_o"], min=50, max=600, step=1, description="D_o [mm]")
t_n        = widgets.FloatSlider(value=defaults["t_n"], min=1, max=30, step=0.1, description="t_n [mm]")
rho_steel  = widgets.FloatText(value=defaults["rho_steel"], description="ρ acero [kg/m³]")
rho_fluid  = widgets.FloatText(value=defaults["rho_fluid"], description="ρ fluido [kg/m³]")
g          = widgets.FloatText(value=defaults["g"], description="g [m/s²]")
E_GPa      = widgets.FloatSlider(value=defaults["E_GPa"], min=100, max=220, step=1, description="E [GPa]")
S_h_MPa    = widgets.FloatSlider(value=defaults["S_h_MPa"], min=50, max=300, step=1, description="S_h [MPa]")
delta_adm  = widgets.FloatSlider(value=defaults["delta_adm"], min=1, max=50, step=1, description="δ_adm [mm]")
w_insul    = widgets.FloatText(value=defaults["w_insul"], description="Peso aislamiento [N/m]")

# Botón reset
reset_btn = widgets.Button(description="🔄 Resetear valores", button_style="warning")

def reset_values(b):
    D_o.value, t_n.value = defaults["D_o"], defaults["t_n"]
    rho_steel.value, rho_fluid.value = defaults["rho_steel"], defaults["rho_fluid"]
    g.value, E_GPa.value = defaults["g"], defaults["E_GPa"]
    S_h_MPa.value, delta_adm.value = defaults["S_h_MPa"], defaults["delta_adm"]
    w_insul.value = defaults["w_insul"]

reset_btn.on_click(reset_values)

# Mostrar controles
display(D_o, t_n, rho_steel, rho_fluid, g, E_GPa, S_h_MPa, delta_adm, w_insul, reset_btn)

# === Función de cálculo ===
def calcular(D_o, t_n, rho_steel, rho_fluid, g, E_GPa, S_h_MPa, delta_adm, w_insul):
    # Conversión a SI
    D_o_m = D_o / 1000
    D_i_m = (D_o - 2*t_n) / 1000
    E     = E_GPa * 1e9
    S_h   = S_h_MPa * 1e6

    # Áreas y pesos lineales
    A_pared = (pi/4) * (D_o_m**2 - D_i_m**2)
    A_fluid = (pi/4) * D_i_m**2
    W_pared = A_pared * rho_steel * g
    W_fluid = A_fluid * rho_fluid * g
    W_total = W_pared + W_fluid + w_insul

    # Propiedades geométricas
    I = (pi/64) * (D_o_m**4 - D_i_m**4)
    Z = I / (D_o_m/2)

    # Spans
    L_sft_m     = sqrt((8 * Z * S_h) / W_total)
    L_flecha_m  = ((384 * E * I * (delta_adm/1000)) / (5 * W_total))**0.25
    L_span_m    = min(L_sft_m, L_flecha_m)

    # Conversión a mm
    L_sft    = L_sft_m * 1000
    L_flecha = L_flecha_m * 1000
    L_span   = L_span_m * 1000

    # Resultados en HTML
    html = f"""
    <div style="border:2px solid #006400; border-radius:10px; padding:15px; background:#f9f9f9; font-family:Arial;">
        <h3 style="color:#006400;">📊 Resultados del cálculo de soporte</h3>
        <ul style="font-size:15px; line-height:1.6;">
            <li>📏 <b>Span por esfuerzo admisible:</b> {L_sft:.1f} mm</li>
            <li>📐 <b>Span por flecha admisible:</b> {L_flecha:.1f} mm</li>
            <li>✅ <b>Span adoptado:</b> {L_span:.1f} mm</li>
        </ul>
    </div>
    """
    display(HTML(html))

# === Conectar widgets ===
out = widgets.interactive_output(
    calcular, {
        "D_o": D_o, "t_n": t_n, "rho_steel": rho_steel, "rho_fluid": rho_fluid,
        "g": g, "E_GPa": E_GPa, "S_h_MPa": S_h_MPa, "delta_adm": delta_adm, "w_insul": w_insul
    }
)
display(out)

FloatSlider(value=219.1, description='D_o [mm]', max=600.0, min=50.0, step=1.0)

FloatSlider(value=8.18, description='t_n [mm]', max=30.0, min=1.0)

FloatText(value=7850.0, description='ρ acero [kg/m³]')

FloatText(value=1000.0, description='ρ fluido [kg/m³]')

FloatText(value=9.81, description='g [m/s²]')

...

FloatSlider(value=120.0, description='S_h [MPa]', max=300.0, min=50.0, step=1.0)

FloatSlider(value=10.0, description='δ_adm [mm]', max=50.0, min=1.0, step=1.0)

FloatText(value=0.0, description='Peso aislamiento [N/m]')

Button(button_style='warning', description='🔄 Resetear valores', style=ButtonStyle())

Output()